## Scrapping


In [1]:
from bs4 import BeautifulSoup

html_doc = 'fafea'

soup = BeautifulSoup(html_doc, 'html.parser')

In [2]:
import re
import requests
from bs4 import BeautifulSoup, Comment

In [3]:
url = f'https://www.marmiton.org/recettes/recette_gratin-pomme-de-terre-et-panais_532213.aspx' # valuepage={page}

In [4]:
response = requests.get(url, headers={"Accept-Language":"en-US"})
soup = BeautifulSoup(response.content, "html.parser")

# print(soup)

In [5]:
# from bs4 import Comment

# for child in soup.body.div.children:
#     if isinstance(child,Comment):
#         child.extract()

In [6]:
ingredients_list = []
for ingredients in soup.find_all("div", class_="fLWRho"):
    title = ingredients.find_all("span", class_="epviYI") #.find("a").string
#     duration = int(movie.find(class_="runtime").string.strip(' min'))
#     year = int(re.search(r"\d{4}", movie.find(class_="lister-item-year").string).group(0))
    ingredients_list.append(title)
#     movies.append({'title': title, 'duration': duration, 'year': year})
print(ingredients_list[1][0])
# ingredients_list[1].content
soup2 = BeautifulSoup(str(ingredients_list[1][0]), "html.parser")


# from bs4 import Comment

# if isinstance(soup2.string,Comment):
#     souped2 = soup2.extract()
#     print(souped2)
num=0
for element in soup2(text=lambda text: isinstance(text, Comment)):
    print(element.extract())

    

<span class="SHRD__sc-10plygc-0 epviYI">1<!-- --> <!-- -->branche</span>
 
 


In [7]:
print(str(soup2))

<span class="SHRD__sc-10plygc-0 epviYI">1 branche</span>


In [8]:
soupstr=str(soup2)
result=re.sub(r'<!.*?->','', soupstr)


In [10]:
# BeautifulSoup(soup2.string)

In [11]:
units = ['branche', 'bouquet','cl','cuillère à café bien bombée', 'c.à.c', 'c.à.s' 'g', \
         'gousse', 'gousses', 'ml', 'plaques', 'pincée', 'poignée', 'pot', 'tablette', 'tranches', 'verre', 'verres']
doubts = ['1⁄2', '1⁄4']

In [12]:
# 1⁄2 bouquet
# de persil

## Prep nettoyage de données

In [13]:
import pandas as pd
ingredients_df = pd.read_csv('../createur_de_recette/data/ingredients.csv')
recipes_df = pd.read_csv('../createur_de_recette/data/recipes.csv')
recipes_df.head(2)

,recipe_id,recipe_url,recipe_title,recipe_type,recipe_steps
0,10534,https://www.marmiton.org/recettes/recette_le-c...,Le crumble aux pommes du Chat qui Tousse,Dessert,Préchauffer le four à 210°C (thermostat 7). \n...
1,10531,https://www.marmiton.org/recettes/recette_esca...,Escalopes lucullus à la cocotte,Plat principal,"Coupez l'oignon en petits morceaux, faites rev..."


In [14]:
recipes_df.shape

(70603, 5)

In [15]:
recipes_df['first_letter'] = recipes_df.recipe_steps.apply(lambda x : x[0])

In [16]:
recipes_df.first_letter.value_counts().sort_values()[0:49]

à       1
:       1
`       1
¨       1
'       1
➢       1
\n      2
u       2
8       2
0       2
[       2
t       2
~       2
_       2
°       3
n       3
5       3
6       3
h       4
o       4
b       4
a       5
>       5
v       5
Z       6
•       6
e       6
s       7
"       7
3       7
r       8
<      11
4      13
.      16
À      17
é      24
l      24
       32
(      32
m      38
d      44
2      44
Q      50
*      62
c      63
p      70
f      71
Ô      72
J     100
Name: first_letter, dtype: int64

In [17]:
# recipes_df.first_letter.value_counts().sort_values()[0:49]
recipes_df.first_letter.value_counts().sort_values()[46:80]

f       71
Ô       72
J      100
U      148
G      202
I      244
O      345
N      688
H      790
-      803
1      843
V      915
T      930
A     1074
S     1158
B     1342
R     1488
É     2166
L     3672
E     4434
M     6429
D     7320
C     7661
F    10659
P    16401
Name: first_letter, dtype: int64

In [18]:
DELETE_ROW_IF_FIRST_CHAR=[':','`', '¨', '➢']
REPLACE_IF_FIRST_CHAR = [' ', '-', '\n']
STR_TO_REPLACE_BY_SINGLE_OCCURENCES = ['!', '.', ' ', '-', ]



In [19]:
recipes_df.shape


(70603, 6)

In [20]:
def recipe_df_cleaning(recipes_df) :
    """Function dedicated to the cleaning of the recipe's DataFrame cooking steps
    
    Dropped :
        - steps with HTML Tags in it '<' 
        - steps starting with ':','`', '¨', '➢', '\n' ( only 1 encounters as first character in steps)
        
    Replaced characters :
        - if first char is ' ' or '-'
        
        - multiples !!! ... and spaces '    '        

    """
    # Creating a column with the first letters of each recipe_steps to identify troublemakers
    recipes_df['first_letter'] = recipes_df.recipe_steps.apply(lambda x : x[0])
    
    # Deleting HTML Tags (balises)
    rows_to_drop = list(recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : x.find('<')) != -1].index)
    recipes_df.drop(rows_to_drop, axis=0, inplace=True)
    
    recipes_df = recipes_df.reset_index().drop(columns='index')
    
    # Deleting rows with first_letter in DELETE_ROW_IF_FIRST_CHAR
    for deleted_char in DELETE_ROW_IF_FIRST_CHAR :
        rows_to_drop = list(recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : \
                                x.startswith(deleted_char))].index)
        
    recipes_df.drop(rows_to_drop, axis=0, inplace=True)
    recipes_df = recipes_df.reset_index().drop(columns='index')
    
    # Replacing first chars of rows if it starts with a char from REPLACE_IF_FIRST_CHAR (Twice, just in cases '- ' or ' .')
    for i in range(2) :
        for replaced_char in REPLACE_IF_FIRST_CHAR :
            recipes_df.recipe_steps = recipes_df.recipe_steps.apply(lambda x : x[1:] \
                    if x.startswith(replaced_char) else x)
            
            
    # Replacing multiples occurences like !!!!! to ! or ... to . or '        ' to ' '
    for i in range(5) : 
        for char in STR_TO_REPLACE_BY_SINGLE_OCCURENCES:
            recipes_df.recipe_steps = recipes_df.recipe_steps.apply(lambda x : x.replace(f"{char}{char}", f"{char}"))
        
    
    # dropping useless columns
    recipes_df.drop(columns='first_letter', inplace=True)
    
    return recipes_df

def drop_rows(df, rows_to_drop, index ) : 
    """ delete rows and reset index
    df : DataFrame
    rows_to_drop : list
    
    """
    df.drop(rows_to_drop, axis=0, inplace=True)
    df = df.reset_index().drop(columns='index')
    return df

trans_recipe = recipe_df_cleaning(recipes_df)
trans_recipe

,recipe_id,recipe_url,recipe_title,recipe_type,recipe_steps
0,10534,https://www.marmiton.org/recettes/recette_le-c...,Le crumble aux pommes du Chat qui Tousse,Dessert,Préchauffer le four à 210°C (thermostat 7). \n...
1,10531,https://www.marmiton.org/recettes/recette_esca...,Escalopes lucullus à la cocotte,Plat principal,"Coupez l'oignon en petits morceaux, faites rev..."
2,10529,https://www.marmiton.org/recettes/recette_terr...,Terrine de foie de porc,Plat principal,"hacher (pas trop fin) le foie, le lard, les éc..."
3,10530,https://www.marmiton.org/recettes/recette_lapi...,Lapin au vin blanc (à la cocotte),Plat principal,Faire revenir les échalotes et les lardons dan...
4,10528,https://www.marmiton.org/recettes/recette_truf...,Truffes en chocolat,Confiserie,La veille : faire fondre le chocolat au bain-m...
...,...,...,...,...,...
70397,532439,https://www.marmiton.org/recettes/recette_buch...,Bûche de Noël au chocolat et crêpes dentelles,Dessert,Faire fondre les chocolats au bain marie avec ...
70398,532434,https://www.marmiton.org/recettes/recette_cook...,Cookies aux noix de pécan et au chocolat,Dessert,Pour le beurre de pécan : Faire fondre le beur...
70399,532401,https://www.marmiton.org/recettes/recette_sand...,Sando au poulet,Plat principal,Couper le poulet en morceaux de 5/6cm. \n\n Ba...
70400,532213,https://www.marmiton.org/recettes/recette_grat...,Gratin pomme de terre et panais,Accompagnement,Éplucher les pommes de terre et le panais et l...


In [21]:
stringed = 'twto'
stringed.startswith('t')
70401 - trans_recipe.recipe_steps[trans_recipe.recipe_steps.apply(lambda x : x.find('.')) != -1].count()

306

In [22]:
recipes_df.recipe_steps[recipes_df.first_letter == '.']
# recipes_df.recipe_steps[recipes_df.first_letter == '.'].reset_index().recipe_steps[0]

41117    . Préchauffer le four à 200°, thermostat 6-7. ...
42276    . Eplucher les 3 gousses d'ail. Les couper en ...
47476    . Coupez le saumon en petits dés. Ciselez de l...
47504    . Pâte : mélangez dans un saladier les ingrédi...
47906    . Verser les amandes dans le lait. Casser les ...
47907    . Eplucher les oignons, les hacher, et les fai...
48148    . couper le curé nantais en tranches \n\n . ba...
48503    . Amalgamer la farine, la levure, la pincée de...
48504    . La veille, découper la viande de sanglier en...
48562    . Laver les fruits. Blanchir le citron. Dans u...
48704    . Si vous avez acheté votre pâte en grande sur...
49519    .Décortiquer les crevettes et les faire marine...
50068    . Préchauffer le four à 100°C au maximum.  \n\...
50331    . Faites préchauffer le four Th 6 ou 180 °C. \...
Name: recipe_steps, dtype: object

In [23]:
recipes_df[recipes_df.recipe_steps.apply(lambda x : x.find("<"))!= -1].count()

recipe_id       0
recipe_url      0
recipe_title    0
recipe_type     0
recipe_steps    0
first_letter    0
dtype: int64

In [24]:
recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : x.find('<')) != -1]

Series([], Name: recipe_steps, dtype: object)

In [25]:
list(recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : x.find('<')) != -1])[2]

IndexError: list index out of range

In [26]:
recipes_df_new = recipes_df.replace(to_replace =' {2,}', value = ' ', regex = True)

In [27]:
list(recipes_df_new.recipe_steps[recipes_df_new.recipe_steps.apply(lambda x : x.find('aa')) != -1])

["Battez l'oeuf dans une assiette creuse et élatez la noix de coco râpée dans une autre assiette. \n\n Trempez les tranches d'ananas épongées, d'abord dans l'oeuf, puis dans la noix de coco. Recommencez l'opération. Laissez reposer au frais recouvert d'une feuille de papier film. \n\n Allumez le four à thermostat 6 (180°C). \n\n Lavez et épépinez les poivrons. Taillez-les en grandes lanières. Coupez le jambon en cubes et les tomates en morceaux. \n\n Dans une grande sauteuse, faites revenir 3mn (sans coloration) l'oignon haché dans l'huile chaude. Ajoutez les poivrons. Poursuivez la cuisson 5 mn et versez le riz en pluie. \n\n Remuez 5 mn puis mettez les tomates, le thym, le laurier, l'ail haché et le poivre de Cayenne. Arrosez de bouillon, portez à ébullition, ajoutez les dés de jambon et un peu de sel. \n\n Couvrez et faites cuire au four 25 à 30 mn. \n\n Laissez reposer 5 mn dans le four éteint, porte entr'ouverte. \n\n Faites dorer les rondelles d'ananas panées dans 30g de beurre c

In [28]:
recipes_df = recipes_df.replace(to_replace =' {2,}', value = ' ', regex = True)

In [29]:
recipes_df['steps_length'] = recipes_df.recipe_steps.apply(lambda x : len(x))
# recipes_df[recipes_df.steps_length<30].shape
# recipes_df[recipes_df.steps_length>30 &  recipes_df.steps_length<100].recipe_steps
# recipes_df[recipes_df.steps_length>30] & recipes_df[recipes_df.steps_length<100]


list(recipes_df.query('steps_length >= 40 & steps_length < 50').recipe_steps)


['Mélanger simplement tous les ingrédients !',
 'Une simple émulsion et le tour est joué !',
 "Il suffit de mélanger le tout dans l'ordre donné.",
 'Mélanger le tout et laisser macérer 3-4 jours.',
 "Préparation : Mélanger le tout. C'est prêt!",
 "48h a l'avance, melanger les indredients",
 'Mixer tous les ingrédients et servir aussitôt.',
 "C'est très simple : tout mélanger dans un mixeur.",
 'Mélanger tout les ingrédients et boire trés frais',
 'Mélanger tous les ingrédients dans un bol.',
 'Mettre le tout dans un mixeur et dégustez.',
 'Mettre dans un bol, mélanger, et bon appétit!!!',
 'Mélanger le tout. \n\n Ne pas laisser reposer.',
 'Mixer tous les ingrédients et réserver au frais.',
 "Mélangez tous les ingrédients... c'est prêt!!!",
 'Mélanger et shaker dans un shaker à glaçons !',
 "Mélanger, verser, c'est prêt à déguster.",
 'Mixer le tout au blender environ 30 sec.',
 'Simplement mélanger le tout et... Santé!',
 'Mélangez tous les ingrédients et dégustez frais !',
 "Mélanger

In [30]:
recipes_df

,recipe_id,recipe_url,recipe_title,recipe_type,recipe_steps,first_letter,steps_length
0,10534,https://www.marmiton.org/recettes/recette_le-c...,Le crumble aux pommes du Chat qui Tousse,Dessert,Préchauffer le four à 210°C (thermostat 7). \n...,P,567
1,10531,https://www.marmiton.org/recettes/recette_esca...,Escalopes lucullus à la cocotte,Plat principal,"Coupez l'oignon en petits morceaux, faites rev...",C,555
2,10529,https://www.marmiton.org/recettes/recette_terr...,Terrine de foie de porc,Plat principal,"- hacher (pas trop fin) le foie, le lard, les ...",-,538
3,10530,https://www.marmiton.org/recettes/recette_lapi...,Lapin au vin blanc (à la cocotte),Plat principal,Faire revenir les échalotes et les lardons dan...,F,286
4,10528,https://www.marmiton.org/recettes/recette_truf...,Truffes en chocolat,Confiserie,La veille : faire fondre le chocolat au bain-m...,L,381
...,...,...,...,...,...,...,...
70598,532439,https://www.marmiton.org/recettes/recette_buch...,Bûche de Noël au chocolat et crêpes dentelles,Dessert,Faire fondre les chocolats au bain marie avec ...,F,733
70599,532434,https://www.marmiton.org/recettes/recette_cook...,Cookies aux noix de pécan et au chocolat,Dessert,Pour le beurre de pécan : Faire fondre le beur...,P,973
70600,532401,https://www.marmiton.org/recettes/recette_sand...,Sando au poulet,Plat principal,Couper le poulet en morceaux de 5/6cm. \n\n Ba...,C,785
70601,532213,https://www.marmiton.org/recettes/recette_grat...,Gratin pomme de terre et panais,Accompagnement,Éplucher les pommes de terre et le panais et l...,É,977
